In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from numpy import linalg as LA

import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import qiskit.tools.visualization as qkviz
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.monitor import job_monitor
from qiskit import BasicAer

from qiskit import execute, Aer, IBMQ

In [2]:
def Ang_theta(Delta0,k_vec):
    kx = k_vec[0]
    ky = k_vec[1]
    Delta = Delta0 + np.cos(kx) + np.cos(ky)
    T = np.sin(kx)
    D = np.sin(ky)
    E = np.sqrt(T**2 + D**2 + Delta**2)
    Inp = np.sqrt(T**2 + D**2) # Inp always larger than 0
    theta = np.arctan2(Inp,Delta)
    return theta

def Ang_varphi(Delta0,k_vec):
    kx = k_vec[0]
    ky = k_vec[1]
    T = np.sin(kx)
    D = np.sin(ky)
    Inp = np.sqrt(T**2 + D**2)
    if Inp<=1e-10:
        varphi = 0
    else:
        varphi = np.arctan2(D,T)
    return varphi

def build_U(vec1,vec2):
    in_product = np.dot(vec1,vec2.conj())

    U = in_product / np.abs(in_product)

    return U

In [3]:
def xlink(Delta0, kx, ky, Dkx):
    Nx = len(kx)
    Ny = len(ky)
    Ux = np.zeros((Nx*Ny),complex)
    for nx in range(Nx):
        for ny in range(Ny):
            k = np.array([kx[nx],ky[ny]], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            psi = np.array([-np.sin(theta/2)*np.exp(1j*varphi),np.cos(theta/2)])
            ############
            k = np.array([kx[nx]+Dkx,ky[ny]], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            psiDx = np.array([-np.sin(theta/2)*np.exp(1j*varphi),np.cos(theta/2)])
            Ux[nx*Ny+ny] = build_U(psi, psiDx )
    return Ux

def ylink(Delta0, kx, ky, Dky):
    Nx = len(kx)
    Ny = len(ky)
    Uy = np.zeros((Nx*Ny),complex)
    for nx in range(Nx):
        for ny in range(Ny):
            k = np.array([kx[nx],ky[ny]], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            psi = np.array([-np.sin(theta/2)*np.exp(1j*varphi),np.cos(theta/2)])
            ############
            k = np.array([kx[nx],ky[ny]+Dky], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            psiDy = np.array([-np.sin(theta/2)*np.exp(1j*varphi),np.cos(theta/2)])
            Uy[nx*Ny+ny] = build_U(psi, psiDy )
    return Uy

In [4]:
N = 8
kx = np.arange(0,N,1)*2*pi/N
ky = np.arange(0,N,1)*2*pi/N
Dkx = (2.*pi)/N
Dky = (2.*pi)/N

Ny = len(ky)

Dlist0=np.linspace(-2.05,-1.95,10)

chernN = np.zeros(len(Dlist0))
for nd, Delta in enumerate(Dlist0):
    Ux = xlink(Delta, kx, ky, Dkx)
    Uy = ylink(Delta, kx, ky, Dky)
    sumNc = 0
    for nx in range(len(kx)):
        for ny in range(len(ky)):
            U1x = Ux[nx*Ny+ny]
            U2y = Uy[nx*Ny+ny]
            if ny+1==Ny:
                U1y = Ux[nx*Ny]
            else:
                U1y = Ux[nx*Ny+(ny+1)]
            if nx+1==Ny:
                U2x = Uy[ny]
            else:
                U2x = Uy[(nx+1)*Ny+ny]
            LF = np.log( U1x * U2x * 1./U1y * 1./U2y)
            sumNc += LF
    chernN[nd] = sumNc.imag/(2.*pi)

In [5]:
def U(qc,qt,theta,phi,lam):
    qc.cx(qt[2],qt[1])
    qc.cu3(theta, phi, lam, qt[1], qt[2])
    qc.cx(qt[2],qt[1])
    
def CCU1(qc,qt,theta,phi,lam):
    qc.cu3(theta, -lam+pi, -phi+pi, qt[1], qt[2])
    qc.cx(qt[0],qt[1])
    qc.cu3(theta, phi, lam, qt[1], qt[2])
    qc.cx(qt[0],qt[1])
    qc.cu3(theta, -lam+pi, -phi+pi, qt[0], qt[2])
    
def CCU(qc,qt,theta,phi,lam):
    qc.cu3(theta, phi, lam, qt[1], qt[2])
    qc.cx(qt[0],qt[1])
    qc.cu3(theta, -lam+pi, -phi+pi, qt[1], qt[2])
    qc.cx(qt[0],qt[1])
    qc.cu3(theta, phi, lam, qt[0], qt[2])
    
def BC_real(qc, qt, c, theta1, phi1, lam1, theta2, lam2, phi2):
    qc.h(qt[0])
    qc.x(qt[2])
    U(qc,qt,theta1,phi1,lam1)
    qc.ccx(qt[0],qt[2],qt[1])
    CCU1(qc,qt,theta1/2,phi1,lam1)
    CCU(qc,qt,theta2/2,phi2,lam2)
    qc.ccx(qt[0],qt[2],qt[1])
    qc.u3(-pi/2,0,0,qt[0])
    qc.barrier(qt)
    qc.measure(qt[0], c[0])
    
def BC_imag(qc, qt, c, theta1, phi1, lam1, theta2, lam2, phi2):
    qc.h(qt[0])
    qc.x(qt[2])
    U(qc,qt,theta1,phi1,lam1)
    qc.ccx(qt[0],qt[2],qt[1])
    CCU1(qc,qt,theta1/2,phi1,lam1)
    CCU(qc,qt,theta2/2,phi2,lam2)
    qc.ccx(qt[0],qt[2],qt[1])
    qc.u3(pi/2,-pi/2,pi/2,qt[0])
    qc.barrier(qt)
    qc.measure(qt[0], c[0])
    
def buildR(theta,thetap,varphi,varphip):
    
    qt = QuantumRegister(3,'sys')
    c = ClassicalRegister(1)
    qc = QuantumCircuit(qt, c)
    
    theta1 = theta
    theta2 = thetap
    lam1 = -varphi
    phi1 = varphi
    lam2 = -varphip
    phi2 = varphip
    
    BC_real(qc, qt, c, theta1, phi1, lam1, theta2, lam2, phi2)

    return qc

def buildI(theta,thetap,varphi,varphip):
    
    qt = QuantumRegister(3,'sys')
    c = ClassicalRegister(1)
    qc = QuantumCircuit(qt, c)
    
    theta1 = theta
    theta2 = thetap
    lam1 = -varphi
    phi1 = varphi
    lam2 = -varphip
    phi2 = varphip
    
    BC_imag(qc, qt, c, theta1, phi1, lam1, theta2, lam2, phi2)

    return qc

def Ux_circuit(Delta0, kx, ky, Dkx):
    Nx = len(kx)
    Ny = len(ky)
    circuits = []
    for nx in range(Nx):
        for ny in range(Ny):
            k = np.array([kx[nx],ky[ny]], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            ### neighbored
            k = np.array([kx[nx]+Dkx,ky[ny]], float)
            thetapx = Ang_theta(Delta0,k)
            varphipx = Ang_varphi(Delta0,k)
            #### real circuit append
            qc = buildR(theta,thetapx,varphi,varphipx)
            circuits.append(qc)
            #### imaginary circuit append
            qc = buildI(theta,thetapx,varphi,varphipx)
            circuits.append(qc)
    return circuits

def Uy_circuit(Delta0, kx, ky, Dky):
    Nx = len(kx)
    Ny = len(ky)
    circuits = []
    for nx in range(Nx):
        for ny in range(Ny):
            k = np.array([kx[nx],ky[ny]], float)
            theta = Ang_theta(Delta0,k)
            varphi = Ang_varphi(Delta0,k)
            ### neighbored
            k = np.array([kx[nx],ky[ny]+Dky], float)
            thetapx = Ang_theta(Delta0,k)
            varphipx = Ang_varphi(Delta0,k)
            #### real circuit append
            qc = buildR(theta,thetapx,varphi,varphipx)
            circuits.append(qc)
            #### imaginary circuit append
            qc = buildI(theta,thetapx,varphi,varphipx)
            circuits.append(qc)
    return circuits

In [6]:
def obtain_result(joblist,nshots,Num):
    result = joblist.result()
    Uc = np.zeros((Num),complex)
    for nx in range(Num):
        Rets= np.zeros(2)
        for n in range(2):
            num = result.get_counts(n+2*nx)
            r=list(num.keys())
            a=list(num.values())
            Num_Sta = np.zeros(2)
            coe = np.zeros(2)
            for j in range(0,len(r)):
                label = [ int(s) for s in r[j] ]
                lab_state = 0
                for m in range(len(label)):
                    lab_state = lab_state + 2**m*label[m]
                Num_Sta[lab_state] = a[j]/nshots
                coe[lab_state] =(-1)**(np.sum(label))
            Rets[n] = np.dot(coe,Num_Sta)
        UR = Rets[0]
        UI = Rets[1]
        Uc[nx] = (UR + 1j*UI) / np.abs(UR + 1j*UI)
    return Uc

In [7]:
def get_Chern(kx,ky,Ux,Uy):
    sumNc = 0
    for nx in range(len(kx)):
        for ny in range(len(ky)):
            U1x = Ux[nx*Ny+ny]
            U2y = Uy[nx*Ny+ny]
            if ny+1==Ny:
                U1y = Ux[nx*Ny]
            else:
                U1y = Ux[nx*Ny+(ny+1)]
            if nx+1==Ny:
                U2x = Uy[ny]
            else:
                U2x = Uy[(nx+1)*Ny+ny]
            LF = np.log( U1x * U2x * 1./U1y * 1./U2y)
            sumNc += LF
    ChernNum = sumNc.imag/(2.*pi)
    return ChernNum

In [ ]:
backend_sim = BasicAer.get_backend('qasm_simulator')
nshots=1024*5

Dlist=np.linspace(-2.05,-1.95,6)

jobx = []
for nd, Delta in enumerate(Dlist):
    gap = Delta
    circx = Ux_circuit(Delta, kx, ky, Dkx)
    qobj=qiskit.transpile(circx,initial_layout=[0,1,2])
    job = qiskit.execute(qobj, backend_sim,shots=nshots)
    jobx.append(job)

In [ ]:
joby = []
for nd, Delta in enumerate(Dlist):
    gap = Delta
    circy = Uy_circuit(Delta, kx, ky, Dky)
    qobj=qiskit.transpile(circy,initial_layout=[0,1,2])
    job = qiskit.execute(qobj, backend_sim,shots=nshots)
    joby.append(job)

In [ ]:
ChernNum = np.zeros(len(Dlist))
for nd, Delta in enumerate(Dlist):
    job = jobx[nd]
    Ux = obtain_result(job,nshots,N**2)
    job = joby[nd]
    Uy = obtain_result(job,nshots,N**2)
    ChernNum[nd] = get_Chern(kx,ky,Ux,Uy)
    print(nd)